# Visualización de datos

marcadores [aquí](https://matplotlib.org/api/markers_api.html?highlight=markers#module-matplotlib.markers)

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import locale

# Cargando y ordenando los datos

incidencia_fc = pd.read_csv('../Datos/IDEFC_NM_may2020.csv', encoding = 'iso-8859-1', thousands = ',')
vars_id = incidencia_fc.columns.values[range(0, 7)]
incidencia_tidy = incidencia_fc.melt(id_vars = vars_id, 
                   var_name = 'Mes', 
                   value_name= 'Total')

locale.setlocale(locale.LC_TIME, 'es_ES')
incidencia_tidy = incidencia_tidy.assign(Periodo = lambda x: x.Año.astype('str') + '-' + x.Mes)
incidencia_tidy['Tiempo'] = pd.to_datetime(incidencia_tidy['Periodo'], format = "%Y-%B")
incidencia_tidy.set_index('Tiempo')

# Funciones para manipular los datos

def q25(series): 
    return np.quantile(series, 0.25)

def q75(series):
    return np.quantile(series, 0.75)

def make_estado(Estado):
    data_estado = (incidencia_tidy
                    .query('Entidad == @Estado')
                    .groupby(['Tiempo'])
                    .agg({'Total':sum})
                    )
    return(data_estado)

def robo_anual_entidad(Entidad):
    data_anual = (incidencia_tidy.query('Entidad == @Entidad and `Subtipo de delito`== "Robo a transeúnte en vía pública"')
                  .groupby(['Tiempo'])
                  .agg({'Total':sum}))
    data_anual = (data_anual.groupby(data_anual.index.strftime('%Y'))
                  .agg([sum, 'mean', 'std', 'min', 'max', q25, q75]))
    return data_anual     

In [2]:
aguas = make_estado('Aguascalientes')
yuca = make_estado('Yucatán')
robo_aguas = robo_anual_entidad('Aguascalientes')
robo_yuca = robo_anual_entidad('Yucatán')

In [3]:
%matplotlib widget
fig, ax = plt.subplots()

ax.plot(aguas['Total'])
ax.plot(yuca['Total'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
%matplotlib widget
fig, ax = plt.subplots()
ax.plot(aguas['Total'], 
        marker = 'o', linestyle = '--')
ax.plot(yuca['Total'],
       marker = 'v', linestyle = '-.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
%matplotlib widget
fig, ax = plt.subplots()
ax.plot(aguas['Total'], 
        marker = 'o', linestyle = '--', color = 'red')
ax.plot(yuca['Total'],
       marker = 'v', linestyle = '-.', color = 'green')

ax.set_xlabel('Tiempo (en meses)')  #Métodos "set_"
ax.set_ylabel('Crímenes (total)')
ax.set_title('Incidencia criminal en Aguascalientes y Yucatán')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
%matplotlib widget
fig, ax = plt.subplots()

# La parte de aguascalientes
ax.plot(robo_aguas[('Total','mean')], marker = 'o', color = 'darkslategrey')
ax.plot(robo_aguas[('Total', 'q25')], marker = '^', color = 'firebrick', linestyle = ':')
ax.plot(robo_aguas[('Total', 'q75')], marker = 'v', color = 'firebrick', linestyle = ':')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
%matplotlib widget
fig, ax = plt.subplots()

# La parte de aguascalientes
ax.plot(robo_aguas[('Total','mean')], marker = 'o', color = 'darkslategrey')
ax.plot(robo_aguas[('Total', 'q25')], marker = '^', color = 'firebrick', linestyle = ':')
ax.plot(robo_aguas[('Total', 'q75')], marker = 'v', color = 'firebrick', linestyle = ':')
# Añadimos yucatán
ax.plot(robo_yuca[('Total','mean')], marker = 'o')
ax.plot(robo_yuca[('Total', 'q25')], marker = '^', linestyle = ':', color = 'orange')
ax.plot(robo_yuca[('Total', 'q75')], marker = 'v', linestyle = ':', color = 'orange')
plt.show()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
%matplotlib widget
fig, ax = plt.subplots(1, 2)
ax[0].plot(robo_aguas[('Total','mean')], marker = 'o', color = 'darkslategrey')
ax[0].plot(robo_aguas[('Total', 'q25')], marker = '^', color = 'firebrick', linestyle = ':')
ax[0].plot(robo_aguas[('Total', 'q75')], marker = 'v', color = 'firebrick', linestyle = ':')

ax[1].plot(robo_yuca[('Total','mean')], marker = 'o')
ax[1].plot(robo_yuca[('Total', 'q25')], marker = '^', linestyle = ':', color = 'orange')
ax[1].plot(robo_yuca[('Total', 'q75')], marker = 'v', linestyle = ':', color = 'orange')

ax[0].set_title('Aguascalientes')
ax[1].set_title('Yucatán')

ax[0].set_ylabel('Robos totales')

fig.suptitle('Robos totales y percentiles del 25 y 75')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Tu turno

Otra forma de arreglar el problema del gráfico anterior es dar a cada curva su propio eje y.  En este ejercicio lo harás siguiendo los siguientes pasos:

1. Define `fig` y `ax` con `plt.subplots()`
2. Añade el gráfico de los robos totales de Aguascalientes
3. Nomina el eje x como `'Tiempo'` y el eje y como `'Aguascalientes (total)'`
4. Define `ax2` con el método `ax.twinx()` para generar un nuevo sistema coordenado.  Este sistema coordenado comparte la coordenada x con ax; pero tiene la coordenada y libre.  
5. En el eje ax2 añade el gráfico de los robos totales en Yucatán
6. Nomina el eje y de ax2 como `'Yucatán (total)'`
7. Agrega un título a tu gráfico

Si lo consideras necesario o útil, da un color distinto a cada línea y usa marcadores.  Si usas colores, añade el argumento `color = ...` al método `.set_ylabel()`

In [10]:
%matplotlib widget
fig, ax = plt.subplots()
ax.plot(robo_aguas[('Total','mean')], marker = 'o', color = 'blue')
ax.set_xlabel('Tiempo')
ax.set_ylabel('Aguascalientes (total)', color = 'blue')
ax.tick_params('y', colors = 'blue')
ax2 = ax.twinx()
ax2.plot(robo_yuca[('Total', 'mean')], color = 'orange', marker = 'o')
ax2.set_ylabel('Yucatán (total)', color = 'orange')
ax2.tick_params('y', colors = 'orange')
ax.set_title('Robos Totales')
plt.show

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<function matplotlib.pyplot.show(*args, **kw)>

In [11]:
# Haciendo lo anterior una función (de una serie)
def add_ts_plot(axis, y, color, ylabel, *args):
    axis.plot(y, color = color, *args)
    axis.set_ylabel(ylabel, color = color)
    axis.tick_params('y', colors = color)

fig, ax = plt.subplots()
add_ts_plot(ax, robo_aguas[('Total', 'mean')], 'blue', 'Aguascalientes (medio)')

ax2 = ax.twinx()
add_ts_plot(ax2, robo_yuca[('Total', 'mean')], 'orange', 'Yucatán (medio)')
    
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
# OTRA COSA UTIL: EL ZOOM POR FECHAS
mis_años = aguas['2016-05-01':'2018-05-31']
fig, ax = plt.subplots()
ax.plot(mis_años['Total'], 
        marker = 'o', linestyle = '--', color = 'darkcyan')
ax.set_xlabel('Tiempo (en meses)')  #Métodos "set_"
ax.set_ylabel('Crímenes (total)')
plt.xticks(rotation=45)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([736055., 736146., 736238., 736330., 736420., 736511., 736603.,
        736695., 736785.]),
 <a list of 9 Text xticklabel objects>)

## Comparaciones y resúmenes

In [13]:
def robos_nacional_año(año):
    robos_nacional = (incidencia_tidy
                 .query('Año == @año and `Subtipo de delito` == "Robo a transeúnte en vía pública"')
                 .groupby(['Entidad', 'Modalidad'])
                 .agg({'Total':sum})).unstack()  
    return robos_nacional

In [14]:
robos_2017 = robos_nacional_año(2017)
robos_2017.head()

Total              
Modalidad           Con violencia Sin violencia
Entidad                                        
Aguascalientes              659.0        1290.0
Baja California            3395.0        2287.0
Baja California Sur          54.0          20.0
Campeche                     42.0           0.0
Chiapas                     976.0         360.0

In [15]:
%matplotlib widget
fig, ax = plt.subplots()

ax.set_title('Robos en el año 2017')

ax.bar(robos_2017.index, robos_2017[('Total', 'Con violencia')])
ax.set_ylabel('Robos Totales en el año')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
%matplotlib widget
fig, ax = plt.subplots()

ax.set_title('Robos en el año 2017')

ax.bar(robos_2017.index, robos_2017[('Total', 'Con violencia')])
ax.set_ylabel('Robos Totales en el año')

ax.set_xticklabels(robos_2017.index, rotation = 90,
                  fontdict = {'fontsize': 5})

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
%matplotlib widget
fig, ax = plt.subplots()

ax.set_title('Robos en el año 2017')

ax.bar(robos_2017.index, robos_2017[('Total', 'Con violencia')])
ax.set_ylabel('Robos Totales en el año')

ax.bar(robos_2017.index, robos_2017[('Total', 'Sin violencia')], bottom = robos_2017[('Total', 'Con violencia')])

ax.set_xticklabels(robos_2017.index, rotation = 90,
                  fontdict = {'fontsize': 5})

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
%matplotlib widget
fig, ax = plt.subplots()

ax.set_title('Robos en el año 2017')

ax.bar(robos_2017.index, robos_2017[('Total', 'Con violencia')], label = 'Con violencia')
ax.set_ylabel('Robos Totales en el año')

ax.bar(robos_2017.index, robos_2017[('Total', 'Sin violencia')], bottom = robos_2017[('Total', 'Con violencia')], label = 'Sin violencia')

ax.set_xticklabels(robos_2017.index, rotation = 90,
                  fontdict = {'fontsize': 5})
ax.legend()

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Tu turno
Repite la idea del gráfico anterior; pero genera un gráfico porcentual (la altura de la barra azul deberá ser el porcentaje de robos con violencia en cada entidad federativa).  Para ello:

1. Genera una nueva base de datos que contenga las columnas porcentuales (tendrás que calcularlas)
2. Sigue el procedimiento del gráfico anterior con estas nuevas columnas

In [19]:
a = (incidencia_tidy
 .query('Año == 2017 and `Subtipo de delito`=="Robo a transeúnte en vía pública"')[['Entidad', 'Modalidad', 'Total']]
 .pivot_table(index = 'Entidad', columns = ['Modalidad'], aggfunc = sum, margins = True, margins_name = 'Total')
)

a.columns = a.columns.get_level_values(1)

a = a.assign(CV = lambda x:x['Con violencia'] / x['Total']).assign(SV = lambda x:x['Sin violencia'] / x['Total'])

%matplotlib widget
fig, ax = plt.subplots()

ax.set_title('Robos en el año 2017')

ax.bar(a.index, a['CV'], label = 'Con violencia')
ax.set_ylabel('Robos Totales en el año')

ax.bar(a.index, a['SV'], bottom = a['CV'], label = 'Sin violencia')

ax.set_xticklabels(a.index, rotation = 90,
                  fontdict = {'fontsize': 5})
ax.legend()

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
robos_2018 = incidencia_tidy.query('`Subtipo de delito`== "Robo a transeúnte en vía pública" and Año == 2018')
robos_2018_cdmx = robos_2018.query('Entidad == "Ciudad de México"')
robos_2018_edomx = robos_2018.query('Entidad == "México"')

In [21]:
%matplotlib widget
fig, ax = plt.subplots()
ax.hist(robos_2018_edomx['Total'], label = 'Estado de México', bins = 20) #key-word argument bins
ax.hist(robos_2018_cdmx['Total'], label = 'Ciudad de México', bins = 30)  #kew-word argument alpha
ax.set_xlabel('Número de robos')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Tu turno
Utiliza un histograma con diferente número de `bins` para investigar el quilataje de los diamantes de la base `diamantes_clean`.

In [ ]:
"""
diamantes = pd.read_csv('../Datos/diamonds_factors.csv', parse_dates = [10])
diamantes['color'] = diamantes['color'].astype('category')
diamantes['price'] = diamantes['price'].str.strip('$').astype('float')
bad_records = diamantes['date_sold'] > dt.datetime.today()
index = diamantes[bad_records].index
diamantes.drop(index = index)
diamantes_clean = diamantes.drop_duplicates(keep = 'last')
diamantes_clean
"""
fig, ax = plt.subplots(2, 2)
bins = np.array([30, 50, 100, 200]).reshape(2, 2)
for i in [0, 1]:
    for j in [0,1]:
        ax[i, j].hist(diamantes['carat'], bins = bins[i, j])
fig.suptitle('Dispersión de la variable `carat`')

In [22]:
%matplotlib widget
fig, ax = plt.subplots()

ax.boxplot([robos_2018_edomx.query('Modalidad == "Con violencia"')['Total'],
            robos_2018_edomx.query('Modalidad == "Sin violencia"')['Total']],
           labels = ['Con Violencia', 'Sin Violencia'])  # kwd-arg vert
ax.set_ylabel('Robos totales')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Robos totales')

## Seaborn

>Seaborn is a Python data visualization library based on matplotlib. It provides a high-level interface for drawing attractive and informative statistical graphics.

Puede ver [el sitio oficial](https://seaborn.pydata.org/index.html)

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt

tips = sns.load_dataset("tips")
tips.head()

In [19]:
%matplotlib widget
sns.relplot(x="total_bill", y="tip", data=tips)

"""
fig, ax = plt.subplots()
ax.scatter(tips.total_bill, tips.tip, edgecolors = 'white')
"""

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

"\nfig, ax = plt.subplots()\nax.scatter(tips.total_bill, tips.tip, edgecolors = 'white')\n"

In [20]:
sns.relplot(x="total_bill", y="tip", hue="smoker", data=tips)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
sns.relplot(x="total_bill", y="tip", hue="smoker", style="smoker",
            data=tips)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
sns.relplot(x="total_bill", y="tip", hue="size", data=tips) #La escala con un float-eable es un gradiente

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Para saber más sobre las paletas de color, puede visitarse [esta página](https://seaborn.pydata.org/tutorial/color_palettes.html).  En variables categóricas tenemos :

1.  Colores con saltos uniformes en el tono con `sns.hls_palette(n, h, l, s)` y `sns.husl_palette(n, h, s, l)` 
2.  Paletas conformadas por el [ColorBrewer](https://colorbrewer2.org/#type=sequential&scheme=BuGn&n=3).  Seaborn cuenta con las paletas `deep`, `muted`, `bright`, `pastel`, `dark`, y `colorblind`. 
3.  Paletas del tipo [Cube Helix](https://jiffyclub.github.io/palettable/cubehelix/) (que también se usan en la representación de escalas continuas de color).

In [42]:
sns.palplot(sns.husl_palette(9, s = 0.8, l = 0.8))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [49]:
sns.palplot(sns.color_palette('Blues', 9))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [53]:
sns.palplot(sns.cubehelix_palette(8))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …